# **Template OP on salish**

In [2]:
%matplotlib inline

import sys
import xarray as xr
import numpy as np
import os
import math
from datetime import datetime, timedelta
from parcels import FieldSet, Field, VectorField, ParticleSet, JITParticle, ErrorCode, ParcelsRandom

sys.path.append('/home/jvalenti/MOAD/analysis-jose/notebooks/parcels')
from Kernels_beaching import DeleteParticle, Buoyancy, AdvectionRK4_3D, Stokes_drift, Beaching, Unbeaching
from OP_functions_beaching import *

In [3]:
# Define paths
local = 0 #Set to 0 when working on server
paths = path(local)


In [14]:
Dat=xr.open_dataset(get_WW3_path(datetime(2018, 12, 23)))
path_NEMO = make_prefix(datetime(2018, 12, 23), paths['NEMO'])
Dat0=xr.open_dataset(path_NEMO + '_grid_W.nc')
Dat=xr.open_dataset(path_NEMO + '_grid_T.nc')
coord=xr.open_dataset(paths['coords'],decode_times=False)
WW3 = xr.open_dataset(get_WW3_path(datetime(2018, 12, 23)))
batt=xr.open_dataset(paths['mask'],decode_times=False)

## Define and save mask for distance to coast

In [15]:
# def maskcoast():
#     maskd=np.zeros(batt.mbathy.shape)
#     for ilat in range(baty.shape[0]):
#         for jlon in range(baty.shape[1]):
#             if baty[ilat,jlon]>0:
#                 if baty[ilat+1,jlon] == 0 or baty[ilat+1,jlon+1] == 0  \
#                  or baty[ilat+1,jlon-1] == 0 or baty[ilat,jlon] == 0   \
#                  or baty[ilat,jlon+1] == 0 or baty[ilat,jlon-1] == 0   \
#                  or baty[ilat-1,jlon] == 0 or baty[ilat-1,jlon+1] == 0 or baty[ilat-1,jlon-1] == 0:
#                     maskd[0,ilat,jlon] = 1
#     return maskd

# baty = batt.mbathy[0,:,:]
# maskd=maskcoast()

# def maskcoast2(maskd):
#     maskd2=np.zeros(batt.mbathy.shape)
#     Ilat=np.where(maskd[0,:,:]==1)[0]
#     Jlon=np.where(maskd[0,:,:]==1)[1]
#     for i in range(len(np.where(maskd[0,:,:]==1)[0])):
#         ilat=Ilat[i]
#         jlon=Jlon[i]
#         maskd2[0,ilat,jlon]=1
#         #if ilat<baty.shape[0] or jlon<baty.shape[1]:
        
#     return maskd2 
# maskd2=maskcoast2(maskd)
# dist=xr.DataArray(attrs={'Distc':maskd2})
# batt['Distc']=(batt.mbathy.dims,maskd)
# batt.to_netcdf(path='/ocean/jvalenti/MOAD/grid/mesh_maskd2T201702.nc')

In [16]:
#clat,clon = p_unidist(coord.gphif[0,:,:],coord.glamf[0,:,:],batt.mbathy[0,:,:],10,10)
with open('clat.txt') as f:
    clat = f.read()
    clat= clat[1:-1]
    clat0 = clat.split(",")
    f.close()
with open('clon.txt') as f:
    clon = f.read()
    clon=clon[1:-1]
    clon0 = clon.split(",")
    f.close()
    
clat,clon=[],[]
for i in range(len(clat0)):
    clat.append(float(clat0[i]))
    clon.append(float(clon0[i]))

## Definitions

In [17]:
start = datetime(2018, 10, 1) #Start date
# Set Time length [days] and timestep [seconds]
length = 50
duration = timedelta(days=length)
dt = 90 #toggle between - or + to pick backwards or forwards
N = len(clat) # number of deploying locations
n = 1 # 1000   # number of particles per location
dmin = list(np.zeros(len(clat))) #minimum depth
dd = 5 #max depth difference from dmin
x_offset, y_offset, zvals = p_deploy(N,n,dmin,dd)

In [18]:
from parcels import Variable

class MPParticle(JITParticle):        
    ro = Variable('ro', initial = 1025)           
    diameter = Variable('diameter', initial = 1.6e-5)
    length = Variable('length', initial = 61e-5)
    Lb = Variable('Lb', initial = 0.26)  #days needed in days for particle to have 67% probability of beaching if in beaching zone (500m)
    Db = Variable('Db', initial = 1000) #Distance at which particles can randomly beach.
    Ub = Variable('Ub', initial = 69)  #days to have 67% probability of unbeaching
    sediment = Variable('sediment', initial = 0)
    beached = Variable('beached', initial = 0)


In [19]:
lon = np.zeros([N,n])
lat = np.zeros([N,n])
for i in range(N):
    lon[i,:]=(clon[i] + x_offset[i,:])
    lat[i,:]=(clat[i] + y_offset[i,:])
z = zvals

In [20]:
#Set start date time and the name of the output file
name = 'Beaching-UnbeachingL' #name output file
daterange = [start+timedelta(days=i) for i in range(length)]
fn =  name + '_'.join(d.strftime('%Y%m%d')+'_1n' for d in [start, start+duration]) + '.nc'
outfile = os.path.join(paths['out'], fn)
print(outfile)

/home/jvalenti/MOAD/results/Beaching-UnbeachingL20181001_1n_20181120_1n.nc


## Simulation

In [21]:
#Fill in the list of variables that you want to use as fields
varlist=['U','V','W','R']
filenames,variables,dimensions=filename_set(start,length,varlist,local)
field_set=FieldSet.from_nemo(filenames, variables, dimensions, allow_time_extrapolation=True)

varlist=['US','VS','WL']
filenames,variables,dimensions=filename_set(start,length,varlist,local)

us = Field.from_netcdf(filenames['US'], variables['US'], dimensions,allow_time_extrapolation=True)
vs = Field.from_netcdf(filenames['VS'], variables['VS'], dimensions,allow_time_extrapolation=True)
wl = Field.from_netcdf(filenames['WL'], variables['WL'], dimensions,allow_time_extrapolation=True)
field_set.add_field(us)
field_set.add_field(vs)
field_set.add_field(wl)
field_set.add_vector_field(VectorField("stokes", us, vs, wl))

filenames,variables,dimensions=filename_set(start,length,['Bathy'],local)
Bth = Field.from_netcdf(filenames['Bathy'], variables['Bathy'], dimensions,allow_time_extrapolation=True)
field_set.add_field(Bth)

# filenames,variables,dimensions=filename_set(start,length,['D'],local)
# Distc = Field.from_netcdf(filenames['D'], variables['D'], dimensions,allow_time_extrapolation=True)
# field_set.add_field(Distc)

In [22]:
# #Load Salish output as fields
#field_set = FieldSet.from_nemo(filenames, variables, dimensions, allow_time_extrapolation=True)
pset = ParticleSet.from_list(field_set, MPParticle, lon=lon, lat=lat, depth=z, time=start+timedelta(hours=2))

k_sink = pset.Kernel(Buoyancy)
k_waves = pset.Kernel(Stokes_drift)
k_beach = pset.Kernel(Beaching)
k_unbeach = pset.Kernel(Unbeaching)


pset.execute(AdvectionRK4_3D + k_sink + k_waves + k_beach + k_unbeach,
             runtime=duration, 
             dt=dt,
             output_file=pset.ParticleFile(name=outfile, outputdt=timedelta(hours=1)),
             recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle})

INFO: Compiled ArrayMPParticleAdvectionRK4_3DBuoyancyStokes_driftBeachingUnbeaching ==> /tmp/parcels-2894/lib1ac38246b603dfd8b112acf9ff7df34b_0.so


Particle 1670 lost !! [5400.0, 2.6409212185849924, 50.439760141647405, -126.02159458221185]
Particle 1678 lost !! [5400.0, 2.9570712389710554, 50.45277214944927, -126.1063240025724]
Particle 1671 lost !! [15480.0, 1.271803160094123, 50.46240700146364, -125.9996683493537]
Correct cell not found for (-124.850299, 48.577779) after 1000000 iterations
Debug info: old particle indices: (yi, xi) 435 0
            new particle indices: (yi, xi) 435 42
            Mesh 2d shape:  898 398
            Relative particle position:  (xsi, eta) -4.3638363519692518e+01 5.1909842156597374e-01
Particle 1283 lost !! [19440.0, 4.882376004767032, 48.581096634550775, -124.84482165233352]


INFO: Temporary output files are stored in /home/jvalenti/MOAD/results/out-VBURZDME.
INFO: You can use "parcels_convert_npydir_to_netcdf /home/jvalenti/MOAD/results/out-VBURZDME" to convert these to a NetCDF file during the run.
N/A% (0 of 4320000.0) |                  | Elapsed Time: 0:00:00 ETA:  --:--:--

Correct cell not found for (-124.818996, 48.542863) after 1000000 iterations
Debug info: old particle indices: (yi, xi) 426 0
            new particle indices: (yi, xi) 426 42
            Mesh 2d shape:  898 398
            Relative particle position:  (xsi, eta) -4.3367401498682483e+01 6.2910411467157956e-01
Particle 1268 lost !! [20430.0, 5.548761630127706, 48.542687759578946, -124.81177331012319]


  0% (16200.0 of 4320000.0) |            | Elapsed Time: 0:00:02 ETA:   0:09:07

...

 80% (3465000.0 of 4320000.0) |########  | Elapsed Time: 0:37:35 ETA:   0:16:07

Correct cell not found for (-124.756468, 48.471004) after 1000000 iterations
Debug info: old particle indices: (yi, xi) 408 1
            new particle indices: (yi, xi) 408 43
            Mesh 2d shape:  898 398
            Relative particle position:  (xsi, eta) -4.4026703084629702e+01 4.0629115154966267e-01
Particle 1025 lost !! [3470850.0, 18.681887221430976, 48.475140114311074, -124.74935799484938]


 80% (3468600.0 of 4320000.0) |########  | Elapsed Time: 0:37:38 ETA:   0:09:58

Correct cell not found for (-124.856313, 48.588055) after 1000000 iterations
Debug info: old particle indices: (yi, xi) 438 0
            new particle indices: (yi, xi) 437 42
            Mesh 2d shape:  898 398
            Relative particle position:  (xsi, eta) -4.3278666660322635e+01 9.7566951144352454e-01
Particle 928 lost !! [3474720.0, 22.192494982820868, 48.591064954201045, -124.85084380326145]


 80% (3488400.0 of 4320000.0) |########  | Elapsed Time: 0:37:50 ETA:   0:09:29